In [1]:
from models import Dim, Clf
from datasets import coil2000

## Dimensionality Reduction

In [2]:
data = coil2000()
dim = Dim(data.train, data.test)
#dim.unpickle_dim('03-01-dim.pkl')
dim.apply()
dim.pickle_dim('03-06-dim.pkl')

50Dim-LLE-k=68-reg=0.001: 100%|██████████| 3/3 [04:56<00:00, 98.69s/it]      


In [3]:
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

xgb_pipe = Pipeline([('mms', MinMaxScaler()),
                     ('xgb', XGBClassifier())])

params = [{'xgb__n_estimators': [10, 20, 50, 100, 200]}]

gs_xgb = GridSearchCV(xgb_pipe,
                      param_grid=params,
                      scoring='accuracy',
                      cv=5)

for key in dim.new_dim.keys():
    if 'KPCA' not in key:
        print(key)
        gs_xgb.fit(dim.new_dim[key][0].T, dim.y_train)
        print(gs_xgb.best_params_)
        # find best model score
        print(gs_xgb.score(dim.new_dim[key][0].T, dim.y_train))

5Dim-SLMVP-Polynomial-Order=5
{'xgb__n_estimators': 10}
0.9463173716985184
5Dim-SLMVP-Linear
{'xgb__n_estimators': 10}
0.9463173716985184
5Dim-SLMVP-Radial-Gammas=0.01
{'xgb__n_estimators': 10}
0.9450289886192828
5Dim-SLMVP-Radial-Gammas=0.1
{'xgb__n_estimators': 10}
0.9461026411853124
5Dim-SLMVP-Radial-Gammas=1
{'xgb__n_estimators': 20}
0.9467468327249302
5Dim-SLMVP-Radial-Gammas=10
{'xgb__n_estimators': 10}
0.9458879106721065
5Dim-PCA
{'xgb__n_estimators': 10}
0.9467468327249302
5Dim-LOL
{'xgb__n_estimators': 10}
0.9448142581060769
5Dim-LPP-k=68
{'xgb__n_estimators': 100}
0.9426669529740176
5Dim-LLE-k=68-reg=0.001
{'xgb__n_estimators': 20}
0.9456731801589006
10Dim-SLMVP-Polynomial-Order=5
{'xgb__n_estimators': 10}
0.9486794073437835
10Dim-SLMVP-Linear
{'xgb__n_estimators': 10}
0.9480352158041657
10Dim-SLMVP-Radial-Gammas=0.01
{'xgb__n_estimators': 10}
0.9495383293966072
10Dim-SLMVP-Radial-Gammas=0.1
{'xgb__n_estimators': 10}
0.9488941378569895
10Dim-SLMVP-Radial-Gammas=1
{'xgb__n_est

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error

model = XGBClassifier(n_estimators=10)
for key in dim.new_dim.keys():
#    if ('PCA' not in key) & ('LOL' not in key) & ('LPP' not in key) & ('LLE' not in key)
    print(key)
    model.fit(dim.new_dim[key][0].T, dim.y_train)
    y_test_pred = model.predict(dim.new_dim[key][1].T)
    print(mean_squared_error(dim.y_test, y_test_pred))

5Dim-SLMVP-Polynomial-Order=5
0.07124463519313305
5Dim-SLMVP-Linear
0.06866952789699571
5Dim-SLMVP-Radial-Gammas=0.01
0.07124463519313305
5Dim-SLMVP-Radial-Gammas=0.1
0.06952789699570816
5Dim-SLMVP-Radial-Gammas=1
0.06952789699570816
5Dim-SLMVP-Radial-Gammas=10
0.07124463519313305
5Dim-PCA
0.06952789699570816
5Dim-KPCA-Linear


XGBoostError: [20:43:10] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/data/data.cc:455: Check failed: this->labels.Size() % this->num_row_ == 0 (2 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000012696d105 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x0000000126a02aaa xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json) + 4282
  [bt] (2) 3   libxgboost.dylib                    0x0000000126a01902 xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (3) 4   libxgboost.dylib                    0x0000000126983509 XGDMatrixSetInfoFromInterface + 233
  [bt] (4) 5   libffi.dylib                        0x00007ff82d9e0882 ffi_call_unix64 + 82
  [bt] (5) 6   ???                                 0x00007ff7bae75740 

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

0.07124463519313305

In [37]:
y_test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
for key in dim.new_dim.keys():
    clf_model.xgboost_model(
        n_estimators_=10, 
        X_tr=dim.new_dim[key][0].T,
        y_tr=dim.y_train,
        X_test=dim.new_dim[key][1].T,
        y_test=dim.y_test)

XGBoostError: [21:23:44] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/data/data.cc:455: Check failed: this->labels.Size() % this->num_row_ == 0 (2 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001307e5105 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000013087aaaa xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json) + 4282
  [bt] (2) 3   libxgboost.dylib                    0x0000000130879902 xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (3) 4   libxgboost.dylib                    0x00000001307fb509 XGDMatrixSetInfoFromInterface + 233
  [bt] (4) 5   libffi.dylib                        0x00007ff8215a6882 ffi_call_unix64 + 82
  [bt] (5) 6   ???                                 0x00007ff7b0fc7380 